In [ ]:
import json
import os

In [ ]:
data_dir = './data/sets/nuscenes/v1.0-mini/'

In [ ]:
[*os.walk(data_dir)]

In [ ]:
data = {}
for _, dirs, files in os.walk(data_dir):
    for file in files:
        with open(os.path.join(data_dir, file), 'r') as f:
            data[file[:-len('.json')]] = json.load(f)

In [ ]:
data

In [ ]:
data.keys()

In [ ]:
sample_tokens = {}
for sd in data['sample_data']:
    sample_token = sd['sample_token']
    if sample_token not in sample_tokens:
        sample_tokens[sample_token] = []
    sample_tokens[sample_token].append(sd)

In [ ]:
first = sorted([*sample_tokens.values()][0], key=lambda x : x['timestamp'])
first[:5]

In [ ]:
token = first[0]['sample_token']
token

In [ ]:
annotation = [*filter(lambda x:x['sample_token'] == token, data['sample_annotation'])]
annotation[:5]

In [ ]:
_annotation = {}
_a = {}
for a in annotation:
    _a[a['token']] = a
for a in annotation:
    if a['next'] not in _a:
        print(a['next'])
    else:
        print("-------------")
for a in annotation:
    if a['prev'] == '':
        c = a
        chain = []
        while c['next'] != '':
            chain.append(c)
            print(c['next'])
            c = _a[c['next']]

In [ ]:
data['calibrated_sensor']